In [1]:
import pandas as pd
import json
import os
import numpy as np
import requests
import sqlalchemy as sql
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from dotenv import load_dotenv
from pathlib import Path
import datetime
%matplotlib inline

In [2]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

True

In [3]:
# Set response URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"
ltc_url = "https://api.alternative.me/v2/ticker/Litecoin/?convert=USD"

In [4]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response,indent=4,sort_keys=True))

{
    "data": {
        "1": {
            "circulating_supply": 18636775,
            "id": 1,
            "last_updated": 1614124952,
            "max_supply": 21000000,
            "name": "Bitcoin",
            "quotes": {
                "USD": {
                    "market_cap": 907459888302,
                    "percent_change_1h": 0.270573511507494,
                    "percent_change_24h": -9.84109293374046,
                    "percent_change_7d": -0.564837730759476,
                    "percentage_change_1h": 0.270573511507494,
                    "percentage_change_24h": -9.84109293374046,
                    "percentage_change_7d": -0.564837730759476,
                    "price": 48960.0,
                    "volume_24h": 117253228768
                }
            },
            "rank": 1,
            "symbol": "BTC",
            "total_supply": 18636775,
            "website_slug": "bitcoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurr

In [5]:
# Using the Python requests library, make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(eth_response,indent=4,sort_keys=True))

{
    "data": {
        "1027": {
            "circulating_supply": 114795640,
            "id": 1027,
            "last_updated": 1614124911,
            "max_supply": 0,
            "name": "Ethereum",
            "quotes": {
                "USD": {
                    "market_cap": 179531726631,
                    "percent_change_1h": 0.115045596554955,
                    "percent_change_24h": -11.5307345098605,
                    "percent_change_7d": -11.6040714894371,
                    "percentage_change_1h": 0.115045596554955,
                    "percentage_change_24h": -11.5307345098605,
                    "percentage_change_7d": -11.6040714894371,
                    "price": 1575.72,
                    "volume_24h": 63182328962
                }
            },
            "rank": 2,
            "symbol": "ETH",
            "total_supply": 114795640,
            "website_slug": "ethereum"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurr

In [6]:
# Using the Python requests library, make an API call to access the current price LTC
ltc_response = requests.get(ltc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(ltc_response,indent=4,sort_keys=True))

{
    "data": {
        "2": {
            "circulating_supply": 66542415,
            "id": 2,
            "last_updated": 1614124943,
            "max_supply": 84000000,
            "name": "Litecoin",
            "quotes": {
                "USD": {
                    "market_cap": 11725725534,
                    "percent_change_1h": 1.12506313159946,
                    "percent_change_24h": -14.6959612208954,
                    "percent_change_7d": -15.588016420451,
                    "percentage_change_1h": 1.12506313159946,
                    "percentage_change_24h": -14.6959612208954,
                    "percentage_change_7d": -15.588016420451,
                    "price": 177.63,
                    "volume_24h": 12056716535
                }
            },
            "rank": 8,
            "symbol": "LTC",
            "total_supply": 66542415,
            "website_slug": "litecoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrencies"

In [7]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response['data']['1']['quotes']['USD']['price']

In [8]:
# Navigate the ETH response object to access the current price of ETH
eth_price = eth_response['data']['1027']['quotes']['USD']['price']

In [9]:
# Navigate the LTC response object to access the current price of LTC
ltc_price = ltc_response['data']['2']['quotes']['USD']['price']

In [10]:
# remember to update date
today_date = str(datetime.datetime.now()).split()[0]
one_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=1*365)).split()[0]
three_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=3*365)).split()[0]
str(datetime.datetime.now()).split(" ")[0]
# Format current date as ISO format
today = pd.Timestamp(today_date, tz="America/New_York").isoformat()
today
# Format start and end dates as ISO format for one year period
start = pd.Timestamp(one_year_ago, tz="America/New_York").isoformat()
end = pd.Timestamp(today_date, tz="America/New_York").isoformat()
# Included for example
# # Get closing prices for FB and TWTR from the last year
# df_portfolio_year = alpaca.get_barset(
#     tickers,
#     timeframe,
#     start = start,
#     end = end
# ).df

In [11]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key,alpaca_secret_key,api_version='v2')
display(type(alpaca_api_key)) # test to make sure api key imported successfully, str indicates import successful
display(type(alpaca_secret_key)) # test to make sure api key imported successfully, str indicates import successful

str

str

In [12]:
# # Set the tickers to be used
# large_company_stocks=['QQQ']

# medium_company_stocks=['ARKK']

# small_company_stocks=[
# 'XSTLA',
# 'CLF',
# 'MSTR',
# 'NEO',
# 'DDD',
# 'OMCL']

# bonds = ['AGG']

In [13]:
tickers = ['AGG','QQQ']

# Set timeframe to 1D 
timeframe = '1D'

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
start_date = pd.Timestamp('2016-02-19',tz='America/New_York').isoformat() # remember to update date
end_date = pd.Timestamp('2021-02-19',tz='America/New_York').isoformat() # remember to update date

In [14]:
# Use the Alpaca get_barset function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
prices_df = alpaca.get_barset(tickers,timeframe,start=start_date,end=end_date).df

# Review the first 5 rows of the Alpaca DataFrame
prices_df.head()

AGG                                         QQQ  \
                             open    high       low   close   volume     open   
2020-09-28 00:00:00-04:00  118.18  118.22  118.1100  118.20  5318716  276.615   
2020-09-29 00:00:00-04:00  118.28  118.33  118.2100  118.23  3982030  276.720   
2020-09-30 00:00:00-04:00  118.26  118.26  117.9516  118.07  5314447  276.090   
2020-10-01 00:00:00-04:00  117.81  118.04  117.7500  117.99  9555596  281.800   
2020-10-02 00:00:00-04:00  118.03  118.04  117.8100  117.92  5367665  276.010   

                                                                 
                               high       low   close    volume  
2020-09-28 00:00:00-04:00  277.0400  271.6740  277.03  51184391  
2020-09-29 00:00:00-04:00  277.7100  275.3719  275.86  23686706  
2020-09-30 00:00:00-04:00  281.0900  275.8717  278.10  44849000  
2020-10-01 00:00:00-04:00  282.8794  279.8400  282.26  44619187  
2020-10-02 00:00:00-04:00  282.2412  273.4400  274.33  66549291

In [15]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
# agg_close_price = prices_df['AGG']['close'].item()

In [16]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
# spy_close_price = prices_df['SPY']['close'].item()

In [17]:
# Access the closing price for QQQ from the Alpaca DataFrame
# Converting the value to a floating point number
# qqq_close_price = prices_df['QQQ']['close'].item()

In [18]:
# Access the closing price for ARKK from the Alpaca DataFrame
# Converting the value to a floating point number
# arkk_close_price = prices_df['ARKK']['close'].item()

In [19]:
# Access the closing price for O from the Alpaca DataFrame
# Converting the value to a floating point number
# o_close_price = prices_df['O']['close'].item()

In [20]:
# Access the closing price for VNQ from the Alpaca DataFrame
# Converting the value to a floating point number
# vnq_close_price = prices_df['VNQ']['close'].item()
# print(vnq_close_price)

In [21]:
# Get user info
user_info = pd.read_csv(Path('data/user_info.csv'))

In [22]:
# Start to build portfolio options
user_info

,current_age,yearly_retirement_income,retirement_age,current_savings,retirement_goal,portfolio_allocation
0,23,50000,65,500000,1250000,"[0.0, 0.7, 0.3]"


In [23]:
age = user_info['current_age']

In [24]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user doesn't want crypto
user_info.loc[:,'portfolio_allocation'] = user_info.loc[:,'portfolio_allocation'].str.replace("'","")
portfolio_allocation = str(user_info['portfolio_allocation'])
print(portfolio_allocation)

0    [0.0, 0.7, 0.3]
Name: portfolio_allocation, dtype: object


In [25]:
# Calculate the % of the portfolio that will be allocated to stocks based on user age if the user does want crypto
if portfolio_allocation == '0    [0.0, 0.7, 0.3]' or '0    [0.2,0.6,0.2]' or '0    [0.4,0.4,0.2]' or '0    [0.5,0.4,0.1]' or '0    [0.7,0.2,0.1]':
    percent_bonds = portfolio_allocation[7:9]
else:
    percent_bonds = portfolio_allocation[6:8]
print(percent_bonds)

.0


In [26]:
if portfolio_allocation == '0    [0.0, 0.7, 0.3]' or '0    [0.2,0.6,0.2]' or '0    [0.4,0.4,0.2]' or '0    [0.5,0.4,0.1]' or '0    [0.7,0.2,0.1]':
    percent_stocks = portfolio_allocation[12:14]
else:
    percent_stocks = portfolio_allocation[10:12]
print(percent_stocks)

.7


In [27]:
if portfolio_allocation == '0    [0.0, 0.7, 0.3]' or '0    [0.2,0.6,0.2]' or '0    [0.4,0.4,0.2]' or '0    [0.5,0.4,0.1]' or '0    [0.7,0.2,0.1]':
    percent_crypto = portfolio_allocation[17:19]
else:
    percent_crypto = portfolio_allocation[14:16]
print(percent_crypto)

.3


In [28]:
stock_amt = float(percent_stocks)*user_info['current_savings']
stock_amt

0    350000.0
Name: current_savings, dtype: float64

In [29]:
btc_amt = float(percent_crypto)*(user_info['current_savings'])
print(btc_amt)

0    150000.0
Name: current_savings, dtype: float64


In [30]:
bond_amt = float(percent_bonds)*(user_info['current_savings'])
print(bond_amt)

0    0.0
Name: current_savings, dtype: float64


In [31]:
if float(stock_amt) > 0:
    stock_weight = (float(stock_amt)+float(bond_amt))/float(stock_amt)
else:
    stock_weight = 0
print(stock_weight)

1.0


In [32]:
if float(bond_amt) > 0:
    bond_weight = (float(bond_amt)+float(stock_amt))/float(bond_amt)
else:
    bond_weight = 0
print(bond_weight)

0


In [33]:
portfolio_weights = [bond_weight,stock_weight]
print(portfolio_weights)

[0, 1.0]


In [34]:
MC_sim = (MCSimulation(portfolio_data=prices_df,weights=portfolio_weights,num_simulation=100,num_trading_days=252*30))
MC_sim.portfolio_data.head()

AGG                                     \
                             open    high       low   close   volume   
2020-09-28 00:00:00-04:00  118.18  118.22  118.1100  118.20  5318716   
2020-09-29 00:00:00-04:00  118.28  118.33  118.2100  118.23  3982030   
2020-09-30 00:00:00-04:00  118.26  118.26  117.9516  118.07  5314447   
2020-10-01 00:00:00-04:00  117.81  118.04  117.7500  117.99  9555596   
2020-10-02 00:00:00-04:00  118.03  118.04  117.8100  117.92  5367665   

                                            QQQ                              \
                          daily_return     open      high       low   close   
2020-09-28 00:00:00-04:00          NaN  276.615  277.0400  271.6740  277.03   
2020-09-29 00:00:00-04:00     0.000254  276.720  277.7100  275.3719  275.86   
2020-09-30 00:00:00-04:00    -0.001353  276.090  281.0900  275.8717  278.10   
2020-10-01 00:00:00-04:00    -0.000678  281.800  282.8794  279.8400  282.26   
2020-10-02 00:00:00-04:00    -0.000593  276.010  282.2412  273.4400  274.33   

                                                  
                             volume daily_return  
2020-09-28 00:00:00-04:00  51184391          NaN  
2020-09-29 00:00:00-04:00  23686706    -0.004223  
2020-09-30 00:00:00-04:00  44849000     0.008120  
2020-10-01 00:00:00-04:00  44619187     0.014959  
2020-10-02 00:00:00-04:00  66549291    -0.028095

In [35]:
MC_sim.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
1,1.000140e+00,9.938917e-01,1.017427e+00,0.996156,9.996251e-01,1.003753e+00,9.946819e-01,0.988390,1.000910,0.998702,...,0.994261,1.024030,9.932689e-01,9.928072e-01,1.008853,0.972060,1.007376,9.997607e-01,9.888088e-01,9.921437e-01
2,1.002105e+00,1.017654e+00,1.015640e+00,1.002049,9.986179e-01,1.019475e+00,9.872323e-01,1.011188,1.030056,1.007940,...,1.020401,1.040594,9.875658e-01,9.863754e-01,1.047919,0.975359,1.017120,9.996076e-01,9.865016e-01,1.009546e+00
3,1.001781e+00,9.924020e-01,1.019051e+00,1.001496,9.947729e-01,1.016445e+00,9.945000e-01,1.043931,1.033295,0.996260,...,1.029149,1.045656,9.819377e-01,9.903554e-01,1.041702,0.988542,1.020696,9.951729e-01,9.966747e-01,1.021525e+00
4,9.991730e-01,9.981993e-01,9.942657e-01,0.986693,9.888483e-01,1.008776e+00,9.763328e-01,1.054304,1.032144,1.013891,...,1.031595,1.032521,9.861806e-01,9.825199e-01,1.016000,0.984302,1.018786,9.963163e-01,1.016459e+00,1.037836e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7556,1.174406e+06,3.094573e+06,1.035911e+06,393803.712645,1.343364e+06,6.681575e+06,5.644860e+06,74832.341559,303544.408484,894648.321173,...,789176.017367,52789.686929,1.069206e+06,3.424328e+06,283592.086995,236116.659184,28960.256550,3.707311e+06,1.690527e+06,2.134757e+06
7557,1.171755e+06,3.107410e+06,1.021638e+06,394431.041516,1.337965e+06,6.885062e+06,5.690791e+06,76505.225909,300494.227882,895158.908719,...,804316.514705,53119.551476,1.065703e+06,3.423012e+06,286607.603226,241023.392161,28507.478800,3.768241e+06,1.708259e+06,2.142961e+06
7558,1.166070e+06,3.106911e+06,1.016280e+06,391913.821280,1.368363e+06,6.949036e+06,5.634744e+06,77849.925286,299772.995671,887142.316406,...,810861.981174,52766.630010,1.049583e+06,3.481774e+06,290631.431629,235657.945214,28878.432371,3.708480e+06,1.710257e+06,2.159968e+06
7559,1.163427e+06,3.155900e+06,1.018915e+06,389766.636337,1.374554e+06,7.051689e+06,5.580976e+06,77017.228475,303380.754941,889472.121654,...,805299.125766,53452.660798,1.045753e+06,3.471499e+06,291364.847839,239378.786709,29091.478309,3.728228e+06,1.707144e+06,2.114418e+06


In [36]:
cumulative_returns = MC_sim.summarize_cumulative_return()
print(cumulative_returns)

count           1.000000e+02
mean            1.441935e+06
std             1.892449e+06
min             2.928502e+04
25%             3.918733e+05
50%             8.150231e+05
75%             1.515194e+06
max             1.118260e+07
95% CI Lower    6.392804e+04
95% CI Upper    6.377525e+06
Name: 7560, dtype: float64


In [55]:
mean_return = cumulative_returns['mean']
print(mean_return)

1441934.7457417934


In [56]:
std_dev = cumulative_returns['std']
print(std_dev)

1892449.0749815942


In [37]:
prices_df

AGG                                     \
                               open    high       low   close   volume   
2020-09-28 00:00:00-04:00  118.1800  118.22  118.1100  118.20  5318716   
2020-09-29 00:00:00-04:00  118.2800  118.33  118.2100  118.23  3982030   
2020-09-30 00:00:00-04:00  118.2600  118.26  117.9516  118.07  5314447   
2020-10-01 00:00:00-04:00  117.8100  118.04  117.7500  117.99  9555596   
2020-10-02 00:00:00-04:00  118.0300  118.04  117.8100  117.92  5367665   
...                             ...     ...       ...     ...      ...   
2021-02-12 00:00:00-05:00  116.6700  116.74  116.5418  116.58  3209765   
2021-02-16 00:00:00-05:00  116.2200  116.23  116.0200  116.03  5543350   
2021-02-17 00:00:00-05:00  116.2200  116.29  116.1450  116.28  6038386   
2021-02-18 00:00:00-05:00  116.1400  116.28  116.0300  116.20  3910324   
2021-02-19 00:00:00-05:00  116.0187  116.06  115.7515  115.83  4416681   

                               QQQ                                         
                              open      high       low    close    volume  
2020-09-28 00:00:00-04:00  276.615  277.0400  271.6740  277.030  51184391  
2020-09-29 00:00:00-04:00  276.720  277.7100  275.3719  275.860  23686706  
2020-09-30 00:00:00-04:00  276.090  281.0900  275.8717  278.100  44849000  
2020-10-01 00:00:00-04:00  281.800  282.8794  279.8400  282.260  44619187  
2020-10-02 00:00:00-04:00  276.010  282.2412  273.4400  274.330  66549291  
...                            ...       ...       ...      ...       ...  
2021-02-12 00:00:00-05:00  333.710  336.6200  332.6300  336.455  18450250  
2021-02-16 00:00:00-05:00  337.270  338.1900  334.3400  335.510  24387052  
2021-02-17 00:00:00-05:00  332.470  335.6570  330.1700  333.940  28492624  
2021-02-18 00:00:00-05:00  330.230  333.8661  328.3600  332.440  28750896  
2021-02-19 00:00:00-05:00  333.860  333.9900  329.9700  331.040  36173201  

[100 rows x 10 columns]

In [38]:
qqq_close = prices_df['QQQ']['close']

In [39]:
stock_daily_returns = qqq_close.pct_change().dropna()

In [40]:
stock_cumulative_returns = (1 + stock_daily_returns).cumprod()

In [41]:
stock_std_dev = stock_cumulative_returns.std()
stock_std_dev

0.06619335921500948

In [42]:
annualized_stock_std_dev = stock_std_dev * np.sqrt(252)
annualized_stock_std_dev

1.0507870015612844

In [43]:
avg_annual_stock_returns = stock_daily_returns.mean()*252
avg_annual_stock_returns

0.475475286731655

In [44]:
stock_sharpe_ratio = avg_annual_stock_returns/annualized_stock_std_dev
stock_sharpe_ratio

0.45249445037403635

In [45]:
agg_close = prices_df['AGG']['close']

In [46]:
bond_daily_returns = agg_close.pct_change().dropna()

In [47]:
bond_cumulative_returns = (1 + bond_daily_returns).cumprod()

In [48]:
bond_std_dev = bond_cumulative_returns.std()
bond_std_dev

0.004332105541455871

In [49]:
annualized_bond_std_dev = bond_std_dev * np.sqrt(252)
annualized_bond_std_dev

0.0687700434958623

In [50]:
avg_annual_bond_returns = bond_daily_returns.mean()*252
avg_annual_bond_returns

-0.05115421454294733

In [51]:
bond_sharpe_ratio = avg_annual_bond_returns/annualized_bond_std_dev
bond_sharpe_ratio

-0.7438444407269443

In [52]:
portfolio_sharpe_ratio = (stock_weight*stock_sharpe_ratio)+(bond_weight*bond_sharpe_ratio)
portfolio_sharpe_ratio

0.45249445037403635

In [53]:
btc_price

48960.0

In [54]:
btc_owned = btc_price/btc_amt
print(btc_owned)

0    0.3264
Name: current_savings, dtype: float64


In [64]:
user_output = [portfolio_sharpe_ratio,mean_return,btc_owned]
# output1 = [portfolio_sharpe_ratio]
# output2 = [mean_return]
# output3 = [btc_owned]
# output = [output1,output2,output3]
print(user_output)

[0.45249445037403635, 1441934.7457417934, 0    0.3264
Name: current_savings, dtype: float64]


In [67]:
# df_output = pd.DataFrame(user_output,columns=['Sharpe Ratio','Expected Return','Number of Bitcoins Owned'])
# df_output.head()
df = pd.DataFrame(user_output).transpose()
df.head()

,0,1,2
0,0.452494,1.44193e+06,"0 0.3264 Name: current_savings, dtype: float64"


In [68]:
df.columns = ['Sharpe Ratio','Expected Return','Number of Bitcoins Owned']

In [69]:
df.head()

,Sharpe Ratio,Expected Return,Number of Bitcoins Owned
0,0.452494,1.44193e+06,"0 0.3264 Name: current_savings, dtype: float64"


In [71]:
df.to_csv('user_output.csv', sep=',')